In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize

import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

Feature Vector
size of feature vector should be  equal to number of unique vocab


In [2]:
wordnet_lemmatizer = WordNetLemmatizer() #removing s and eds eg: dog and dogs will be the same.

stopwords = set(w.rstrip() for w in open('stopwords.txt'))

In [3]:
positive_reviews = BeautifulSoup(open('electronics/positive.review')) # It's getting HTML page with tags
negative_reviews = BeautifulSoup(open('electronics/negative.review')) 

In [4]:
positive_reviews = positive_reviews.findAll('review_text') # Getting the information from <review_text> tag only
negative_reviews = negative_reviews.findAll('review_text')

In [5]:
print(len(positive_reviews), len(negative_reviews))

1000 1000


In [6]:
np.random.shuffle(positive_reviews) # Run the code with and without shuffle
positive_reviews = positive_reviews[:len(negative_reviews)] # This is generally done assuming we have more positive reveiws than negative reviews

In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/vdna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
# How big is my vocab

def my_tokenizer(s):
    s = s.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    tokens = [t for t in tokens if len(t) > 2] # will remove all the words that has 2 or 1 character
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # Remove the extra characters in the word that has same meaning eg.. dog doggo dogs
    tokens = [t for t in tokens if t not in stopwords] # remove stop words
    return tokens

word_index_map = {}
current_index = 0

positive_tokenized = []
negative_tokenized = []


# Create a dictionary with the information of the words as keys and the count as the value of the corresponding key.
for review in positive_reviews:
    tokens = my_tokenizer(review.text)
    positive_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1
            
            
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [13]:

def tokens_to_vector(tokens, label):
    x = np.zeros(len(word_index_map) + 1)
    for t in tokens:
        i = word_index_map[t]
        x[i] += 1
    x = x / x.sum()
    x[-1] = label
    return x

In [14]:

N = len(positive_tokenized) + len(negative_tokenized)
data = np.zeros((N, len(word_index_map) + 1))
i = 0

In [15]:
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[i,:] = xy
    i += 1
    
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[i,:] = xy
    i += 1

In [26]:
np.random.shuffle(data)

In [27]:
X = data[:, :-1]
Y = data[:, -1]

In [28]:
XTrain = X[:-100,]
YTrain = Y[:-100,]
XTest = X[-100:,]
YTest = Y[-100:,]

In [43]:
model = LogisticRegression()
model.fit(XTrain, YTrain)
print("Classification rate:", model.score(XTest, YTest))

y_pred = model.predict(XTest)

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(YTest, y_pred)

/Users/vdna/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Classification rate: 0.76


In [44]:
threshold = 0.5
cm

array([[38, 13],
       [11, 38]])

In [32]:
for word, index in word_index_map.items():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print(word, weight)

month -0.6940848225936085
wa -1.6350301090818566
returned -0.7186174922897566
home 0.5638609688102857
sound 1.0557049809184174
quality 1.3957617498715775
pretty 0.7581300719915276
memory 0.9365689963631046
using 0.6661514056964634
time -0.6317244335435982
you 0.8974237685222535
then -1.042589355907795
perfect 0.8865304805888062
try -0.6799769771285681
customer -0.6647329173795671
bit 0.6394525286211961
price 2.661032634881903
love 1.13110204082786
've 0.708655331743074
n't -2.040592582424689
ha 0.6059217953423799
little 0.9223445509555118
buy -0.8403640965557254
bad -0.7224359213656332
tried -0.7876087853912271
static -0.501321119501028
poor -0.7921364634737701
doe -1.2821508041761982
lot 0.7391783625489634
money -1.1117215508998581
cable 0.6194154028623847
video 0.5236089100682628
look 0.5637730253173404
week -0.6927445068432616
fast 0.8694183823387399
happy 0.6592196787892529
support -0.9097749861376206
company -0.5501920884844786
recommend 0.5907278840377777
highly 0.950861629303940

In [33]:
# Try different Classifiers
# Try different Features - we uses normallize vector
# Try Regressoin
# Try 5 categories instead of 2
# Different catories of data     